In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
pd.set_option('display.max_rows', 10000)

In [44]:
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']


#changing xls to csv for each file:
"""for i in range(len(list_daerah)):
    df = pd.read_excel(list_daerah[i]+'.xls')
    df = df.loc[[0, 7, 9, 12, 14, 16, 18, 21, 24, 28]].reset_index(drop=True)
    cols = (df != '-').any()
    df = df[cols[cols].index]
    df.to_csv(list_daerah[i]+'.csv')
"""
all_data_frame = {}
drop_index = [1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 15, 17, 19, 20, 22, 23, 25, 26, 27, 29, 30]
for i in list_daerah:
    all_data_frame[i] = pd.read_csv(i +'.csv')
    #all_data_frame[i] = all_data_frame[i].loc[[0, 7, 9, 12, 14, 16, 18, 21, 24, 28]].reset_index(drop=True)
    #all_data_frame[i].drop(drop_index)#.reset_index(drop=True)
    all_data_frame[i] = all_data_frame[i].drop(columns='Unnamed: 0')
    all_data_frame[i] = all_data_frame[i].drop(columns='No.')
    all_data_frame[i].rename(columns={'Komoditas(Rp)': 'Komoditas'}, inplace=True)
    #all_data_frame[i] = all_data_frame[i][all_data_frame[i] != '-']
    #all_data_frame[i] = all_data_frame[i].drop(['17/04/2019', '05/06/2019','06/06/2019', '07/06/2019' , '25/10/2019', ], 1)
    #deleting entire columns that the values is '-'
    #all_data_frame[i] = all_data_frame[i].loc[:, ~(all_data_frame[i] == '-').all()]
    #cols = (all_data_frame[i] != '-').any()
    #all_data_frame[i] = all_data_frame[i][cols[cols].index]
    
#print(all_data_frame['aceh'])
#for i in list_daerah:
    #print(all_data_frame[i].dtypes)
#setelah dicek manual dengan capek sangat, dimana setiap frame di print dtypes nya, dan di search yang mana type object
#ditemukan iregular data bahwa tanggal 20 agustus 2019, di papua barat tidak ada harga daging sapi, 
#sementara bahan pokok lain ada, oleh sebab itu untuk sementara 
#kolom tanggal 20/08/2019 akan dihapus di papua barat
all_data_frame['papua_barat'] = all_data_frame['papua_barat'].drop('20/08/2019', 1)
#print(all_data_frame['kalsel'])

all_data_frame_melt = {}
for i in list_daerah:
    all_data_frame_melt[i] = pd.melt(frame=all_data_frame[i], id_vars=['Komoditas'], var_name='Tanggal', value_name='Harga')
    #change to datetime
    all_data_frame_melt[i]['Tanggal'] = pd.to_datetime(all_data_frame_melt[i]['Tanggal'], dayfirst=True, utc=False)
    all_data_frame_melt[i] = all_data_frame_melt[i].sort_values(by=['Komoditas', 'Tanggal']).reset_index(drop=True)
    all_data_frame_melt[i]['Harga'] = pd.to_numeric(all_data_frame_melt[i]['Harga'], downcast='integer')
#print(all_data_frame_melt['kalsel'])

all_data_frame_pivot = {}
for i in list_daerah:
    all_data_frame_pivot[i] = all_data_frame_melt[i].pivot(columns = 'Komoditas', index='Tanggal', values = 'Harga').reset_index()
    #all_data_frame_pivot[i]['Hari'] = all_data_frame_pivot[i].Tanggal.str.split('/', expand=True)[0]
    all_data_frame_pivot[i]['Hari'] = all_data_frame_pivot[i]['Tanggal'].dt.day
    #all_data_frame_pivot[i]['Bulan'] = all_data_frame_pivot[i].Tanggal.str.split('/', expand=True)[1]
    all_data_frame_pivot[i]['Bulan'] = all_data_frame_pivot[i]['Tanggal'].dt.month
#print(all_data_frame_pivot['jatim'])  


#create specific data frames first
#.to_numpy convert pandas data frame or series into numpy array
#below is a test:
#all_data_frame_commodity_bawang_putih = {}
#for i in all_data_frame_pivot:
    #all_data_frame_commodity_bawang_putih[i] = all_data_frame_pivot[i]['Bawang Putih'].to_numpy()
#print(all_data_frame_commodity_bawang_putih)


#now try to create 10 commodity dictionary and then put them inside of 1 dictionary
#the structure would be dictionary(dict of different commodity) > dictionary(numpy array per province)
commodity_all = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
#all_dict_names = ['Bawang_Merah_all', 'Bawang_Putih_all', 'Beras_all', 'Cabai_Merah_all',
       #'Cabai_Rawit_all', 'Daging_Ayam_all', 'Daging_Sapi_all', 'Gula_Pasir_all',
       #'Minyak_Goreng_all', 'Telur_Ayam_all']

all_data_frame_commodity = {}
for i in range(len(commodity_all)):
    temp_dict = {}
    for a in all_data_frame_pivot:
        temp_dict[a] = all_data_frame_pivot[a][commodity_all[i]].to_numpy()
    #print(all_dict_names[i] )   
    all_data_frame_commodity[commodity_all[i]] = temp_dict
#print(all_data_frame_commodity['Gula Pasir']['jambi'])
#success
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']

all_data_frame_commodity_melt = {}
for i in range(len(commodity_all)):
    temp_dict = {}
    label = []
    value = []
    tanggal = []
    for a in range(len(all_data_frame_commodity[commodity_all[i]])):
        label = label + ([list_daerah2[a]]*len(all_data_frame_commodity[commodity_all[i]][list_daerah[a]]))
        value.append(all_data_frame_commodity[commodity_all[i]][list_daerah[a]])
        tanggal = tanggal + all_data_frame_pivot[list_daerah[a]]['Tanggal'].tolist()
    #temp_dict[all_data_frame_commodity[i][a]] = 
    temp_dict = pd.DataFrame({
            "Provinsi": label,
            "Harga": np.concatenate(value),
            "Tanggal" : tanggal
            })
    all_data_frame_commodity_melt[commodity_all[i]] = temp_dict
#print(all_data_frame_commodity_melt['Beras'])


all_data_frame_pivot_commodity = {}
for i in commodity_all:
    all_data_frame_pivot_commodity[i] = all_data_frame_commodity_melt[i].pivot(columns = 'Provinsi', index='Tanggal', values = 'Harga').reset_index()
    #all_data_frame_pivot[i]['Hari'] = all_data_frame_pivot[i].Tanggal.str.split('/', expand=True)[0]
    all_data_frame_pivot_commodity[i]['Hari'] = all_data_frame_pivot_commodity[i]['Tanggal'].dt.day
    #all_data_frame_pivot[i]['Bulan'] = all_data_frame_pivot[i].Tanggal.str.split('/', expand=True)[1]
    all_data_frame_pivot_commodity[i]['Bulan'] = all_data_frame_pivot_commodity[i]['Tanggal'].dt.month
#print(all_data_frame_pivot_commodity['Gula Pasir'])  

In [45]:
#aggregat
all_aggregat_dict = {}
for i in list_daerah:
    df_mean = all_data_frame_melt[i].groupby('Komoditas').Harga.mean().round(2).reset_index()
    df_mean.rename(columns={'Harga': 'Mean'}, inplace=True)
    df_median = all_data_frame_melt[i].groupby('Komoditas').Harga.median().reset_index()
    df_median.rename(columns={'Harga': 'Median'}, inplace=True)
#df_mode = df_melt.groupby('Komoditas').apply(lambda x: mode(x)[0][0]).reset_index()
#df_mode.rename(columns={'Harga': 'Mode'}, inplace=True)
    df_std = all_data_frame_melt[i].groupby('Komoditas').Harga.std().round(2).reset_index()
    df_std.rename(columns={'Harga': 'STD'}, inplace=True)
    df_max = all_data_frame_melt[i].groupby('Komoditas').Harga.max().reset_index()
    df_max.rename(columns={'Harga': 'Max'}, inplace=True)
    df_min = all_data_frame_melt[i].groupby('Komoditas').Harga.min().reset_index()
    df_min.rename(columns={'Harga': 'Min'}, inplace=True)
#df_agregat = df_melt.groupby('Komoditas').Harga.mean().reset_index()
    df_quartile_1 = all_data_frame_melt[i].groupby('Komoditas').Harga.quantile(q=0.25).reset_index()
    df_quartile_1.rename(columns={'Harga': 'First Quartile'}, inplace=True)
    df_quartile_2 = all_data_frame_melt[i].groupby('Komoditas').Harga.quantile(q=0.50).reset_index()
    df_quartile_2.rename(columns={'Harga': 'Second Quartile'}, inplace=True)
    df_quartile_3 = all_data_frame_melt[i].groupby('Komoditas').Harga.quantile(q=0.75).reset_index()
    df_quartile_3.rename(columns={'Harga': 'Third Quartile'}, inplace=True)
    all_df_aggregate = df_mean.merge(df_median).merge(df_std).merge(df_max).merge(df_min).merge(df_quartile_1).merge(df_quartile_2).merge(df_quartile_3)
    all_df_aggregate['No.'] = range(1, len(all_df_aggregate)+1)
    all_df_aggregate = all_df_aggregate[['No.', 'Komoditas', 'Mean', 'Median', 'Max', 'Min', 'STD', 'First Quartile', 'Second Quartile', 'Third Quartile']]
    all_aggregat_dict[i] = all_df_aggregate
#print(type(all_aggregat_dict['jambi']))

In [46]:
#aggregat per commodity
all_aggregat_commodity_dict = {}
for i in commodity_all:
    df_mean = all_data_frame_commodity_melt[i].groupby('Provinsi').Harga.mean().round(2).reset_index()
    df_mean.rename(columns={'Harga': 'Mean'}, inplace=True)
    df_median = all_data_frame_commodity_melt[i].groupby('Provinsi').Harga.median().reset_index()
    df_median.rename(columns={'Harga': 'Median'}, inplace=True)
#df_mode = df_melt.groupby('Komoditas').apply(lambda x: mode(x)[0][0]).reset_index()
#df_mode.rename(columns={'Harga': 'Mode'}, inplace=True)
    df_std = all_data_frame_commodity_melt[i].groupby('Provinsi').Harga.std().round(2).reset_index()
    df_std.rename(columns={'Harga': 'STD'}, inplace=True)
    df_max = all_data_frame_commodity_melt[i].groupby('Provinsi').Harga.max().reset_index()
    df_max.rename(columns={'Harga': 'Max'}, inplace=True)
    df_min = all_data_frame_commodity_melt[i].groupby('Provinsi').Harga.min().reset_index()
    df_min.rename(columns={'Harga': 'Min'}, inplace=True)
#df_agregat = df_melt.groupby('Komoditas').Harga.mean().reset_index()
    all_df_aggregate = df_mean.merge(df_median).merge(df_std).merge(df_max).merge(df_min)
    all_df_aggregate['No.'] = range(1, len(all_df_aggregate)+1)
    all_df_aggregate = all_df_aggregate[['No.', 'Provinsi', 'Mean', 'Median', 'Max', 'Min', 'STD']]
    all_aggregat_commodity_dict[i] = all_df_aggregate
#all_aggregat_commodity_dict['Cabai Merah']

In [47]:
#line chart province
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def line_plot_all(save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(all_data_frame_pivot)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(list_daerah2[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        for i in range(1, 11):
            list_ax[i] = plt.subplot(2, 5, i)
            plt.plot(range(len(all_data_frame_pivot[list_daerah[a]])), all_data_frame_pivot[list_daerah[a]][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
    #list_ax[i].set_xticklabels(range(1, 13))
            plt.ylabel('Harga (Rp)')
            plt.title(columns_df[i])
            plt.xticks(rotation=90)
            plt.xlabel('Jan - Des 2019')
            plt.xticks([])
            plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
            plt.tight_layout()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.88)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+list_daerah2[a] + '.png')
        plt.show()
#line_plot_all()

def line_plot_provinsi(provinsi, save=None):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    fig = plt.figure(figsize=(20, 10))
    for i in range(1, 11):
        list_ax[i] = plt.subplot(2, 5, i)
        plt.plot(range(len(all_data_frame_pivot[provinsi])), all_data_frame_pivot[provinsi][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
    #list_ax[i].set_xticklabels(range(1, 13))
        plt.ylabel('Harga (Rp)')
        plt.title(columns_df[i])
        plt.xticks(rotation=90)
        plt.xlabel('Jan - Des 2019')
        plt.xticks([])
    plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
    plt.tight_layout()
    fig.suptitle(provinsi, fontsize=30)
    fig.tight_layout()
    fig.subplots_adjust(top=0.88)
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+provinsi+'.png')
    plt.show()

#line_plot_provinsi can be called using one of these argumen below:
# 'aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
#'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
#'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
#'sumbar', 'sumsel', 'sumut'
#and to save the figure, put second argumen 'save=True'

#line_plot_provinsi('jambi')

In [48]:
#Line chart compare comodity
list_daerah = [0, 'aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = [0, 'Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def comodity_line_all(save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10',  'ax11', 'ax12', 'ax13', 'ax14',
               'ax15', 'ax16', 'ax17', 'ax18', 'ax19', 'ax20', 'ax21', 'ax22', 'ax23', 'ax24', 'ax25', 'ax26', 'ax27',
               'ax128', 'ax29', 'ax30', 'ax31', 'ax32', 'ax33', 'ax34']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = [0, 'fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9']
    for i in range(1, len(figure_o)):
        figure_o[i] = plt.figure(figsize=(20, 40))
        figure_o[i].suptitle(columns_df[i], fontsize=30)
        #figure_o[i].subplots_adjust(top=0.8)
        for a in range(1, len(list_ax)):
            list_ax[a] = plt.subplot(7, 5, a)
            plt.plot(range(len(all_data_frame_pivot[list_daerah[a]][columns_df[i]])), all_data_frame_pivot[list_daerah[a]][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
            plt.ylabel('Harga (Rp)')
            plt.title(list_daerah2[a])
            #plt.xticks(rotation=90)
            plt.xlabel('Jan - Des 2019')
            plt.xticks([])
            plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
            plt.tight_layout()  
        figure_o[i].tight_layout()
        figure_o[i].subplots_adjust(top=0.96)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+columns_df[i] + '.png')
        plt.show()
#TO SAVE FIGURE, put second argumen 'save=True'
#comodity_line_all()
                    
def comodity_line(komoditas, save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10',  'ax11', 'ax12', 'ax13', 'ax14',
               'ax15', 'ax16', 'ax17', 'ax18', 'ax19', 'ax20', 'ax21', 'ax22', 'ax23', 'ax24', 'ax25', 'ax26', 'ax27',
               'ax128', 'ax29', 'ax30', 'ax31', 'ax32', 'ax33', 'ax34']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    fig = plt.figure(figsize=(20, 40))
    for a in range(1, len(list_ax)):
        list_ax[a] = plt.subplot(7, 5, a)
        plt.plot(range(len(all_data_frame_pivot[list_daerah[a]][komoditas])), all_data_frame_pivot[list_daerah[a]][komoditas])
    #list_ax[i].set_xticks(range(52))
        plt.ylabel('Harga (Rp)')
        plt.title(list_daerah2[a])
        #plt.xticks(rotation=90)
        plt.xlabel('Jan - Des 2019')
        plt.xticks([])
    plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
    plt.tight_layout()
    fig.suptitle(komoditas, fontsize=30)
    fig.tight_layout()
    fig.subplots_adjust(top=0.96)
    if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+komoditas + '.png')
    plt.show()
    #plt.savefig(list_daerah2)
    
#comodity func can be called using one of these argumens:
#'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah','Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
#'Minyak Goreng', 'Telur Ayam'

#comodity_line('Bawang Putih', save=True)

In [49]:
#bar chart
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def bar_chart_all(save=False):
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(all_data_frame_pivot)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(list_daerah2[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        ax = sns.barplot(data=all_data_frame_melt[list_daerah[a]], x='Komoditas', y='Harga', hue='Tanggal')
        plt.ylabel('Harga (Rp)')
        plt.xlabel('Komoditas (365 hari)')
        ax.get_legend().remove()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+list_daerah2[a] + '_bar.png')
        plt.show()
#line_plot_all(save=True)
#bar_chart_all()


def bar_chart_provinsi(provinsi, save=None):
    fig = plt.figure(figsize=(20, 10))
    ax = sns.barplot(data=all_data_frame_melt[provinsi], x='Komoditas', y='Harga', hue='Tanggal')
    fig.suptitle(provinsi, fontsize=30)
    fig.subplots_adjust(top=0.91)
    plt.ylabel('Harga (Rp)')
    plt.xlabel('Komoditas (jan-des 2019)')
    ax.get_legend().remove()
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+provinsi+'_bar.png')
    plt.show()
#bar_chart_provinsi('sulut')

In [50]:
#bar chart compare commodity per province

def all_bar_commodity(save=False):
    commodity_all = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(commodity_all)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(commodity_all[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        ax = sns.barplot(data=all_data_frame_commodity_melt[commodity_all[a]], x='Provinsi', y='Harga', hue='Tanggal')
        plt.ylabel('Harga (Rp)')
        plt.xlabel('Provinsi (jan-des 2019)')
        plt.xticks(rotation=90)
        ax.get_legend().remove()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+ commodity_all[a] + '_bar.png')
        plt.show()

#all_bar_commodity()

def bar_chart_commodity_one(commodity, save=None):
    fig4 = plt.figure(figsize=(20, 10))
    ax4 = sns.barplot(data=all_data_frame_commodity_melt[commodity], x='Provinsi', y='Harga', hue='Tanggal')
    fig4.suptitle(commodity, fontsize=30)
    fig4.subplots_adjust(top=0.91)
    plt.ylabel('Harga (Rp)')
    plt.xlabel('Provinsi (jan-des 2019)')
    plt.xticks(rotation=90)
    ax4.get_legend().remove()
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+commodity+'_bar.png')
    plt.show()
#bar_chart_commodity_one('Bawang Putih')



In [51]:
#histogram
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def hist_plot_all(save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(all_data_frame_pivot)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(list_daerah2[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        for i in range(1, 11):
            list_ax[i] = plt.subplot(2, 5, i)
            plt.hist(all_data_frame_pivot[list_daerah[a]][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
    #list_ax[i].set_xticklabels(range(1, 13))
            plt.ylabel('Harga (Rp)')
            plt.title(columns_df[i])
            plt.xticks(rotation=90)
            plt.xlabel('Jan - Des 2019')
            #plt.xticks([])
            plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
            plt.tight_layout()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.88)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+list_daerah2[a] + '_histogram.png')
        plt.show()
#hist_plot_all()

def hist_provinsi(provinsi, save=None):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    fig = plt.figure(figsize=(20, 15))
    for i in range(1, 11):
        list_ax[i] = plt.subplot(2, 5, i)
        plt.hist(all_data_frame_pivot[provinsi][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
    #list_ax[i].set_xticklabels(range(1, 13))
        plt.ylabel('Harga (Rp)')
        plt.title(columns_df[i])
        plt.xticks(rotation=90)
        plt.xlabel('Jan - Des 2019')
        #plt.xticks([])
    plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
    plt.tight_layout()
    fig.suptitle(provinsi, fontsize=30)
    fig.tight_layout()
    fig.subplots_adjust(top=0.88)
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+provinsi+'_histogram.png')
    plt.show()
#hist_provinsi('jambi')

In [52]:
#histogram per commodity
list_daerah = [0, 'aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = [0, 'Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def hist_commod_all(save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10',  'ax11', 'ax12', 'ax13', 'ax14',
               'ax15', 'ax16', 'ax17', 'ax18', 'ax19', 'ax20', 'ax21', 'ax22', 'ax23', 'ax24', 'ax25', 'ax26', 'ax27',
               'ax128', 'ax29', 'ax30', 'ax31', 'ax32', 'ax33', 'ax34']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = [0, 'fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9']
    for i in range(1, len(figure_o)):
        figure_o[i] = plt.figure(figsize=(20, 40))
        figure_o[i].suptitle(columns_df[i], fontsize=30)
        #figure_o[i].subplots_adjust(top=0.8)
        for a in range(1, len(list_ax)):
            list_ax[a] = plt.subplot(7, 5, a)
            plt.hist(all_data_frame_pivot[list_daerah[a]][columns_df[i]])
    #list_ax[i].set_xticks(range(52))
            plt.ylabel('Harga (Rp)')
            plt.title(list_daerah2[a])
            #plt.xticks(rotation=90)
            plt.xlabel('Jan - Des 2019')
            #plt.xticks([])
            plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
            plt.tight_layout()  
        figure_o[i].tight_layout()
        figure_o[i].subplots_adjust(top=0.96)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+columns_df[i] + '_hist.png')
        plt.show()
#TO SAVE FIGURE, put second argumen 'save=True'
#hist_commod_all()
                    
def hist_commod_one(komoditas, save=False):
    list_ax = [0, 'ax1', 'ax2', 'ax3', 'ax4', 'ax5', 'ax6', 'ax7', 'ax8', 'ax9', 'ax10',  'ax11', 'ax12', 'ax13', 'ax14',
               'ax15', 'ax16', 'ax17', 'ax18', 'ax19', 'ax20', 'ax21', 'ax22', 'ax23', 'ax24', 'ax25', 'ax26', 'ax27',
               'ax128', 'ax29', 'ax30', 'ax31', 'ax32', 'ax33', 'ax34']
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    fig = plt.figure(figsize=(20, 40))
    for a in range(1, len(list_ax)):
        list_ax[a] = plt.subplot(7, 5, a)
        plt.hist(all_data_frame_pivot[list_daerah[a]][komoditas])
    #list_ax[i].set_xticks(range(52))
        plt.ylabel('Harga (Rp)')
        plt.title(list_daerah2[a])
        #plt.xticks(rotation=90)
        plt.xlabel('Jan - Des 2019')
        #plt.xticks([])
    plt.subplots_adjust(top=0.946, bottom=0.059, left=0.059, right=0.989, hspace=0.228, wspace=0.556)
    plt.tight_layout()
    fig.suptitle(komoditas, fontsize=30)
    fig.tight_layout()
    fig.subplots_adjust(top=0.96)
    if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+komoditas + '_hist.png')
    plt.show()
    #plt.savefig(list_daerah2)
    
#comodity func can be called using one of these argumens:
#'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah','Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
#'Minyak Goreng', 'Telur Ayam'

#hist_commod_one('Bawang Putih')

In [53]:
#Boxplot province
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def box_chart_all(save=False):
    columns_df = [0, 'Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(all_data_frame_pivot)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(list_daerah2[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        ax = sns.boxplot(data=all_data_frame_melt[list_daerah[a]], x='Komoditas', y='Harga')
        plt.ylabel('Harga (Rp)')
        plt.xlabel('Komoditas (365 hari)')
        #ax.get_legend().remove()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+list_daerah2[a] + '_box.png')
        plt.show()
#line_plot_all(save=True)
#box_chart_all()


def box_chart_provinsi(provinsi, save=None):
    fig = plt.figure(figsize=(20, 10))
    ax = sns.boxplot(data=all_data_frame_melt[provinsi], x='Komoditas', y='Harga')
    fig.suptitle(provinsi, fontsize=30)
    fig.subplots_adjust(top=0.91)
    plt.ylabel('Harga (Rp)')
    plt.xlabel('Komoditas (jan-des 2019)')
    #ax.get_legend().remove()
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+provinsi+'_box.png')
    plt.show()
#box_chart_provinsi('sulut')

In [54]:
#box chart commodity
def all_box_commodity(save=False):
    commodity_all = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for a in range(len(commodity_all)):
        figure_o[a] = plt.figure(figsize=(20, 10))
        figure_o[a].suptitle(commodity_all[a], fontsize=30)
        figure_o[a].subplots_adjust(top=0.8)
        ax = sns.boxplot(data=all_data_frame_commodity_melt[commodity_all[a]], x='Provinsi', y='Harga')
        plt.ylabel('Harga (Rp)')
        plt.xlabel('Provinsi (jan-des 2019)')
        plt.xticks(rotation=90)
        #ax.get_legend().remove()
        
        figure_o[a].tight_layout()
        figure_o[a].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+ commodity_all[a] + '_box.png')
        plt.show()

#all_box_commodity()

def box_chart_commodity_one(commodity, save=None):
    fig4 = plt.figure(figsize=(20, 10))
    ax4 = sns.boxplot(data=all_data_frame_commodity_melt[commodity], x='Provinsi', y='Harga')
    fig4.suptitle(commodity, fontsize=30)
    fig4.subplots_adjust(top=0.91)
    plt.ylabel('Harga (Rp)')
    plt.xlabel('Provinsi (jan-des 2019)')
    plt.xticks(rotation=90)
    #ax4.get_legend().remove()
    if save == True:
        plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+commodity+'_box.png')
    plt.show()
#box_chart_commodity_one('Bawang Putih')


In [55]:
#Heatmap
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']
def heatmap_all(save=False):
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9', 'fig10', 'fig11', 'fig12', 'fig13',
            'fig14', 'fig15', 'fig16', 'fig17', 'fig18', 'fig19', 'fig20', 'fig21', 'fig22', 'fig23', 'fig24', 'fig25', 'fig26', 'fig1',
            'fig27', 'fig28', 'fig29', 'fig30', 'fig31', 'fig32', 'fig33']
    for i in range(len(list_daerah)):
        figure_o[i] = plt.figure(figsize=(20, 10))
        c= all_data_frame_pivot[list_daerah[i]][['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah', 'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir','Minyak Goreng', 'Telur Ayam']].corr()
        sns.heatmap(c,cmap="BrBG",annot=True)
        figure_o[i].suptitle(list_daerah2[i], fontsize=30)
        figure_o[i].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+list_daerah2[i]+'_heatmap.png')
        plt.show()
#heatmap_all()

def heatmap_province(prov, save=False):
        fig = plt.figure(figsize=(20, 10))
        c= all_data_frame_pivot[prov][['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah', 'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir','Minyak Goreng', 'Telur Ayam']].corr()
        sns.heatmap(c,cmap="BrBG",annot=True)
        fig.suptitle(prov, fontsize=30)
        fig.subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+provinsi+'_heatmap.png')
        plt.show()
#heatmap_province('jambi')


In [56]:
#Heatmap commodity
commodity_all = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']
def heatmap_commodity_all(save=False):
    figure_o = ['fig', 'fig1', 'fig2', 'fig3', 'fig4', 'fig5', 'fig6', 'fig7', 'fig8', 'fig9']
    for i in range(len(commodity_all)):
        figure_o[i] = plt.figure(figsize=(20, 10))
        c= all_data_frame_pivot_commodity[commodity_all[i]].corr()
        sns.heatmap(c,cmap="BrBG",annot=True)
        figure_o[i].suptitle(commodity_all[i], fontsize=30)
        figure_o[i].subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+commodity_all[i]+'_heatmap.png')
        plt.show()
#heatmap_commodity_all()
def heatmap_commodity(comm, save=False):
        fig = plt.figure(figsize=(20, 10))
        c= all_data_frame_pivot_commodity[comm].corr()
        sns.heatmap(c,cmap="BrBG",annot=True)
        fig.suptitle(comm, fontsize=30)
        fig.subplots_adjust(top=0.91)
        if save == True:
            plt.savefig('E:\\for github\harga pangan indo 2019\savefig\\'+comm+'_heatmap.png')
        plt.show()
#heatmap_commodity('Cabai Rawit')